In [ ]:
import os
import pandas as pd
import seaborn as sns
sns.set_context('poster')
sns.set_style('white')

import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib as mpl
mpl.rcParams['pdf.fonttype'] = 42

import statsmodels.api as sm
import statsmodels.formula.api as smf
import scipy.stats as st

from allensdk.core.mouse_connectivity_cache import MouseConnectivityCache

In [ ]:
path = r'/Users/jenniferwh/Dropbox (Allen Institute)/Mesoscale Connectome Papers in Progress/2018 Plaque/hAPP immunostaining'

In [ ]:
dat = pd.read_csv(os.path.join(path, 'compiled_APP+.csv'))

In [ ]:
dat.head()

In [ ]:
dat['Genotype'].unique()

In [ ]:
no2576 = dat[dat['Genotype'].isin(['APP/PS1', 'J20'])]

In [ ]:
def get_glm(region):
    formula = '%s ~ Genotype + Sex' % region
    results = smf.ols(formula=formula, data=no2576).fit()
    
    return(results)

In [ ]:
res = get_glm('PIR')
anova = sm.stats.anova_lm(res, typ=2)
anova

In [ ]:
res = get_glm('GU')
anova = sm.stats.anova_lm(res, typ=2)
anova

In [ ]:
res = get_glm('CLA')
anova = sm.stats.anova_lm(res, typ=2)
anova

In [ ]:
res_dg = get_glm('ACAv')
anova_dg = sm.stats.anova_lm(res_dg, typ=2)
anova_dg

In [ ]:
get_glm('EPv').summary()

In [ ]:
dat.head()

In [ ]:
mdf = pd.melt(dat, id_vars = ['Genotype', 'Sex'], var_name = 'Region', value_name = 'hAPP_level')

In [ ]:
mdf.tail()

In [ ]:
print(len(mdf))
mdf = mdf[~mdf['hAPP_level'].isnull()]
print(len(mdf))

In [ ]:
formula = 'hAPP_level ~ Genotype + Sex + Region'
results = smf.ols(formula=formula, data=mdf).fit()
anova = sm.stats.anova_lm(results, typ=2)
anova

In [ ]:
meandat = dat.groupby(['Genotype', 'Sex']).mean().reset_index()

In [ ]:
meandat.to_csv(r'/Users/jenniferwh/Dropbox (Allen Institute)/Mesoscale Connectome Papers in Progress/2018 Plaque/hAPP immunostaining/hAPP quantification means.csv')

In [ ]:
meandat = dat.groupby(['Genotype']).mean().reset_index()
meandat.head()

In [ ]:
meandat.

In [ ]:
happ_dat = pd.DataFrame({'region': meandat.columns, 'APP/PS1': meandat.iloc[0].values, 
                         'hAPP-J20': meandat.iloc[1].values,
                        'T2576': meandat.iloc[2].values})

In [ ]:
happ_dat.drop(index = 0, inplace=True)

In [ ]:
happ_dat.head()

In [ ]:
unionizes = pd.read_csv(r'/Users/jenniferwh/Dropbox (Allen Institute)/Mesoscale Connectome Papers in Progress/2018 Plaque/unionizes/plaque_unionizes.csv')

In [ ]:
unionizes = unionizes[unionizes['control'] == False]

In [ ]:
unionizes.head()

In [ ]:
for structure in happ_dat['region']:
    happ_dat.loc[happ_dat['region'] == structure, 
                 'APP/PS1_plaque'] = unionizes[(unionizes['mouse_line'] == 'APP/PS1') & 
                                               (unionizes['structure_acronym'] == structure)]['plaque_density'].mean()
    happ_dat.loc[happ_dat['region'] == structure, 
                 'Tg2576_plaque'] = unionizes[(unionizes['mouse_line'] == 'Tg2576') & 
                                               (unionizes['structure_acronym'] == structure)]['plaque_density'].mean()
    happ_dat.loc[happ_dat['region'] == structure, 
                 'hAPP-J20_plaque'] = unionizes[(unionizes['mouse_line'] == 'hAPP-J20') & 
                                               (unionizes['structure_acronym'] == structure)]['plaque_density'].mean()

In [ ]:
happ_dat.head()

In [ ]:
path

In [ ]:
happ_dat.to_csv(os.path.join(path, 'means_per_line.csv'))

In [ ]:
fig, ax = plt.subplots()
sns.catplot('APP/PS1', 'APP/PS1_plaque', data=happ_dat, ax = ax, color = mouse_line_colors[0])
sns.catplot('T2576', 'Tg2576_plaque', data=happ_dat, ax = ax, color = mouse_line_colors[2])
sns.catplot('hAPP-J20', 'hAPP-J20_plaque', data=happ_dat, ax = ax, color = mouse_line_colors[1])

In [ ]:
path

In [ ]:
fig, ax = plt.subplots(figsize = (14, 5))
h = sns.pointplot('Region', 'hAPP_level', hue = 'Sex', data = mdf[mdf['Genotype'].isin(['APP/PS1', 'hAPP-J20'])], 
                                                                  ax = ax, legend = False,
                 palette = ['hotpink', 'cornflowerblue'])
h.legend_.remove()
h.set_title('APP (+) mice')
h.set_ylabel('hAPP Expression Level')
h.set_xlabel('')
plt.tight_layout()
plt.savefig(os.path.join(path, 'hAPP expression by sex second batch.png'), 
            bbox_inches='tight', pad_inches=0.3, format='png', dpi=300)

In [ ]:
mouse_line_colors = ['#8da0cb', '#fc8d62', '#66c2a5', '#000000']
mouse_lines = ['APP/PS1', 'Tg2576', 'hAPP-J20']

In [ ]:
mdf.loc[mdf['Genotype'] == 'Tg2576 (1)', 'Genotype'] = 'Tg2576'

In [ ]:
fig, ax = plt.subplots(figsize = (15, 5))
g = sns.pointplot('Region', 'hAPP_level', hue = 'Genotype', data = mdf, ax = ax, legend = False,
                 palette = mouse_line_colors)
g.set_title('APP (+) mice')
g.set_ylabel('hAPP Expression Level')
g.set_xlabel('')

plt.tight_layout()
plt.savefig(os.path.join(path, 'hAPP expression by genotype.png'), 
            bbox_inches='tight', pad_inches=0.3, format='png', dpi=300)

In [ ]:
fig, ax = plt.subplots(2, 2, figsize = (15, 10))
h = sns.pointplot('Region', 'hAPP_level', hue = 'Sex', data = mdf[mdf['APP'] == '-'], ax = ax[0, 0], legend = False,
                 palette = ['hotpink', 'cornflowerblue'])
g = sns.pointplot('Region', 'hAPP_level', hue = 'Sex', data = mdf[mdf['APP'] == '+'], ax = ax[0, 1], legend = False,
                 palette = ['hotpink', 'cornflowerblue'])
g.legend(bbox_to_anchor = (1.1, 1))
h.legend_.remove()
h.set_title('APP (-) mice')
h.set_ylabel('hAPP Expression Level')
h.set_xlabel('')
g.set_title('APP (+) mice')
g.set_ylabel('hAPP Expression Level')
g.set_xlabel('')
plt.tight_layout()
i = sns.pointplot('Region', 'hAPP_level', hue = 'Genotype', data = mdf[mdf['APP'] == '-'], ax = ax[1, 0], legend = False,
                 palette = mouse_line_colors)
j = sns.pointplot('Region', 'hAPP_level', hue = 'Genotype', data = mdf[mdf['APP'] == '+'], ax = ax[1, 1], legend = False,
                 palette = mouse_line_colors)
j.legend(bbox_to_anchor = (1.1, 1))
i.legend_.remove()
i.set_title('APP (-) mice')
i.set_ylabel('hAPP Expression Level')
i.set_xlabel('')
j.set_title('APP (+) mice')
j.set_ylabel('hAPP Expression Level')
j.set_xlabel('')
plt.tight_layout()
plt.savefig(os.path.join(path, 'hAPP expression by genotype and sex in hippocampal regions.png'), 
            bbox_inches='tight', pad_inches=0.3, format='png', dpi=300)

In [ ]:
density = pd.read_csv(r'/Users/jenniferwh/Dropbox (Allen Institute)/Mesoscale Connectome Papers in Progress/2018 Plaque/hAPP immunostaining/mean APP and plaque density levels.csv')

In [ ]:
density.drop(columns = {'APP'}, inplace = True)
density.head()

In [ ]:
h = sns.scatterplot('hAPP_expression', 'relative_plaque_density_same_region', hue = 'Genotype', data = density)
plt.legend(bbox_to_anchor = (1.1, 1))

In [ ]:
h = sns.scatterplot('hAPP_expression', '18_mo_relative_pd', hue = 'Genotype', data = density)
plt.legend(bbox_to_anchor = (1.1, 1))

In [ ]:
h = sns.scatterplot('hAPP_expression', 'relative_plaque_density_input_region', hue = 'Genotype', 
                    data = density)

In [ ]:
h = sns.scatterplot('hAPP_expression', '18_mo_relative_input_pd', hue = 'Genotype', 
                    data = density)
plt.legend(bbox_to_anchor = (1.1, 1))

In [ ]:
st.pearsonr(density['hAPP_expression'], density['relative_plaque_density_same_region'])

In [ ]:
st.pearsonr(density[density['Genotype'] == 'J20']['hAPP_expression'], 
            density[density['Genotype'] == 'J20']['plaque_density_same_region'])

In [ ]:
st.pearsonr(density['hAPP_expression'], density['relative_plaque_density_input_region'])

In [ ]:
st.pearsonr(density[density['Genotype'] == 'J20']['hAPP_expression'], 
            density[density['Genotype'] == 'J20']['plaque_density_input_region'])

In [ ]:
st.pearsonr(density[(density['Genotype'] == 'Tg2576') &
                   (density['region'] != 'SUBd')]['hAPP_expression'], 
                                             density[(density['Genotype'] == 'Tg2576') &
                                                     (density['region'] != 'SUBd')][
                                                 '18_mo_relative_input_pd'])

In [ ]:
st.pearsonr(density[density['Genotype'].isin(['J20'])]['hAPP_expression'], 
                                             density[density['Genotype'].isin(['J20'])][
                                                 'plaque_density_same_region'])

In [ ]:
st.pearsonr(density[(density['Genotype'] == 'J20') &
                   (density['region'] != 'SUBd')]['hAPP_expression'], 
                                             density[(density['Genotype'] == 'J20') &
                                                     (density['region'] != 'SUBd')][
                                                 'plaque_density_input_region'])

In [ ]:
st.pearsonr(density['hAPP_expression'][density['Genotype'].isin(['APP/PS1'])], 
            density[density['Genotype'].isin(['APP/PS1'])]['plaque_density_same_region'])

In [ ]:
st.pearsonr(density[(density['Genotype'] == 'APP/PS1') &
                   (density['region'] != 'SUBd')]['hAPP_expression'], 
            density[(density['Genotype'] == 'APP/PS1') &
                   (density['region'] != 'SUBd')]['plaque_density_input_region'])

In [ ]:
st.pearsonr(density['hAPP_expression'], 
            density['plaque_density_input_region'].sample(frac = 1))